In [5]:
# Importing modules...
import numpy as np
import socket		
import time

In [6]:
print('Creating server socket...')

# Try to create server socket
try: 
    server_socket = socket.socket(socket.AF_INET,    # Internet
                                  socket.SOCK_DGRAM) # UDP
    print('> Socket successfully created')
    
except socket.error as err: 
    print(f'> Socket creation failed with error {err}')

UDP_IP = '127.0.0.1' # Server IP
UDP_PORT = 5005      # UDP Port
buffer_size = 1024   # In bytes

server_socket.bind((UDP_IP, UDP_PORT))
print(f'> Server bound to {UDP_PORT}')

print()
while True:
    time_start = time.time()
    message_bytes, address = server_socket.recvfrom(buffer_size)
    message = message_bytes.decode() # From bytes to string
    time_end = time.time()

    if not message:
        break

    if time_end - time_start != 0:
        print(1/(time_end - time_start))
    else:
        print('VERY FAST')

    # print(f'Received message from IP {address[0]}, Port {address[1]}:')
    # print(f'> {message}')

print()
print('Connection Endend.')

Creating server socket...
> Socket successfully created
> Server bound to 5005

0.21531409863237716
59.976891837785274
161.00975047984645
VERY FAST
9.989197016318792
86.88356292076644
101.4808255304735
131.24425808874147
15.603577327718337
262.2751375687844
VERY FAST
45.939802847754656
17.238306214628892
VERY FAST
59.9820381546206
2264.742980561555
16.56001484528919
84.17395492584639
VERY FAST
VERY FAST
15.570155281924116
91.96220044289504
160.81837352862237
66.91080800829545
20.265373075194837
66.55723761464978
81.98885782981802
140.39981254602665
12.679274486094316
386.8213594023794
96.39197481212511
VERY FAST
6.835043599985008
64.01562881562882
43.04057465366855
171.07737488273443
9.28070664560183
107.05490186069068
74.02975801754417
VERY FAST
11.826067342231847
57.46409097136594
82.75400520874437
246.62221438231316
11.988486856208608
80.95392870239911
145.43860744131212
74.63041582889984
12.667977468702336
75.70262611677646
301.9222574143392
78.71896700574302
13.391133885669587
55.